# Icheck
Some helper cells to check what delensalot produced


Check file names and directory paths

In [ ]:
from IPython.display import clear_output
from datetime import datetime

import os
from os.path import join as opj
import numpy as np
import healpy as hp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
print(plt.style.available)
plt.style.use('seaborn-pastel')
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from plancklens import qresp, qest, utils as pl_utils

import delensalot
from delensalot.run import run
from delensalot.utils import cli
from delensalot.core import cachers
import delensalot.core.power.pospace as pospace
from delensalot.utility import plot_helper as ph
from delensalot.utility.utils_hp import Alm, almxfl, alm_copy, gauss_beam, alm2cl
from delensalot.core.iterator.statics import rec as rec
from delensalot.utility.plot_helper import movavg, phi2kappa_bp, bandpass_alms
from delensalot.config.config_helper import data_functions as df

from delensalot.utility import plot_helper
pcmap = plot_helper.get_planck_cmap()

i2cp = {0: r"${\rm fg}_{00}$", 1: r"${\rm fg}_{12}$", 2:r'${\rm fg}_{13}$'}
ik2it =  {'p_p': 'EE+EB', 'pee': 'EE', 'peb': 'EB', 'p':'TT+EE+EB', 'ptt':'TT'}

bpl, bpu = (5, 4000)
def scale_and_bandpass(data):
    ll = np.arange(0,4001)
    return ph.bandpass_alms(hp.almxfl(data,np.sqrt(ll*(ll+1))), bpl, bpu)

def bp(data, lower, upper):
    return ph.bandpass_alms(data, lower, upper)

def status_sg(anas):
    out = {}
    itmax_loc = anas.itmax
    buffer = []
    for simidx in anas.simidxs:
        test = [rec.maxiterdone(anas.libdir_MAP(anas.k, simidx, anas.version)) >= itmax_loc]
        if np.all(test):
            buffer.append(itmax_loc)
        else:
            buffer.append(rec.maxiterdone(anas.libdir_MAP(anas.k, simidx, anas.version)))
    print(buffer, np.sum(1+np.array(buffer)))
    out.update({i2cp[0]: np.array(buffer)})
    pdf = pd.DataFrame(out)
    pdf.plot.bar(xlabel='simidx', ylabel='Rec. iteration', figsize=(16, 2), ylim=(0,20), title='Status analyses: {}'.format(whoami.split('.')[0][13:-3]))
    plt.scatter(anas.simidxs, np.ones_like(anas.simidxs)*itmax_loc, color='black', marker='_')
    plt.grid(which='both', axis='y', ls= '--', color='grey')

    ax = plt.gca()
    ax.yaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    plt.show()

# import pandas as pd
def status(anas):
    out = {}
    itmax_loc = anas[0].itmax
    for anai, ana in enumerate([*anas]):
        buffer = []
        for simidx in anas[0].simidxs:
            test = [rec.maxiterdone(ana.libdir_MAP(ana.k, simidx, ana.version)) >= itmax_loc]
            if np.all(test):
                buffer.append(itmax_loc)
            else:
                buffer.append(rec.maxiterdone(ana.libdir_MAP(ana.k, simidx, ana.version)))
        out.update({i2cp[anai]: np.array(buffer)})
    pdf = pd.DataFrame(out)
    pdf.plot.bar(xlabel='simidx', ylabel='Rec. iteration', figsize=(16, 2), ylim=(0,anas[0].itmax+2), title='Status analyses, {}'.format(anas[0].whoami))
    plt.scatter(anas[0].simidxs, np.ones_like(anas[0].simidxs)*itmax_loc, color='black', marker='_')
    plt.grid(which='both', axis='y', ls= '--', color='grey')

    ax = plt.gca()
    ax.yaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    plt.show()

In [ ]:
fgs = ["11","12","13"]
# fgs = ["00"]
# fgs = ["11"]
PSCRATCH_PATH = '/pscratch/sd/s/sebibel/analysis'
anas = []
anass = []
fn = opj(os.getcwd(), 'config/conf_cscomp_08b.{fg}_mfsimsub_pixwin2.py')
for fi, fg in enumerate(fgs):
    delensalot_runner = run(config_fn=fn.format(fg=fg), job_id='MAP_lensrec', verbose=False)
    # delensalot_runner.run()
    anas.append(delensalot_runner.init_job())
    anas[fi].fg = fg
    # print(fg)
    anas[fi].whoami = 'cscomp - fg {fg}'.format(fg=anas[fi].fg)
       
    anas[fi].ll = np.arange(0,anas[fi].lm_max_qlm[0]+1,1)
    anas[fi].scale_factor_L = (anas[fi].ll*(anas[fi].ll+1))**2/(2*np.pi)
    bpl, bpu = (5, anas[fi].lm_max_qlm[0])
anass.append(anas)
clear_output(wait=True)
status(anas)
print("Cell finished {}".format(datetime.now().strftime("%H:%M:%S")))

ana = anas[0]
lmax_qlm = ana.lm_max_qlm[0]

In [ ]:
for anasi, anas in enumerate(anass):
    LL = np.arange(0,anas[0].lm_max_qlm[0]+1,1)
    scale_factor_L = (LL*(LL+1))**2/(2*np.pi)
    for anai, ana in enumerate(anas):
        # ana.simidxs = [1,2]
        # ana.itmax = 12
        # if anai == 0:
        ana.mask = hp.ud_grade(hp.read_map(ana.mask_fn), nside_out=1024)
        ana.plms_in = np.array([alm_copy(hp.read_alm(opj(os.environ["CFS"],'cmb/data/generic/cmb/ffp10/mc/scalar/ffp10_unlensed_scl_cmb_000_tebplm_mc_%04d.fits'% simidx), hdu=4), None, 4000, 4000) for simidx in ana.simidxs])
        ana.plms = np.array([ana.get_plm_it(simidx, np.arange(ana.itmax)) for simidx in ana.simidxs])

In [ ]:
for anasi, anas in enumerate(anass):
    for anai, ana in enumerate(anas):
        # if anai == 0:
            ana.mfvarmap = np.load(ana.mfvar)
            ana.R = qresp.get_response(ana.k, ana.lm_max_ivf[0]+200, ana.k[0], ana.cls_len, ana.cls_len, ana.ftebl_len, lmax_qlm=ana.lm_max_qlm[0])[0]
            ana.WF = ana.cpp * pl_utils.cli(ana.cpp + pl_utils.cli(ana.R))
            ana.mf = alm_copy(ana.mfvarmap, None, ana.lm_max_qlm[0], ana.lm_max_qlm[1])
            almxfl(ana.mf, 1/ana.R, ana.lm_max_qlm[1], True) # Normalized QE
            ana.mf = np.nan_to_num(ana.mf)

In [ ]:
for anasi, anas in enumerate(anass):
    for anai, ana in enumerate(anas):
        fig, ax = plt.subplots(1,4, figsize=(22,8))
        plt.axes(ax[0])
        hp.mollview(hp.alm2map(scale_and_bandpass(ana.plms[0,0]), nside=1024), title='cscomp.{fg} - '.format(fg=ana.fg)+r'$\hat{\alpha}^{\rm QE}$', cmap='YlGnBu', hold=True, min=-0.002, max=0.002)
        plt.axes(ax[1])
        hp.mollview(hp.alm2map(scale_and_bandpass(ana.plms[0,-1]), nside=1024), title='cscomp.{fg} - '.format(fg=ana.fg)+r'$\hat{\alpha}^{\rm MAP}$', cmap='YlGnBu', hold=True, min=-0.002, max=0.002)
        plt.axes(ax[2])
        hp.mollview(hp.alm2map(scale_and_bandpass(anass[0][0].mfvarmap), nside=1024)*anass[0][0].mask, title='cscomp.{fg} - '.format(fg=ana.fg)+r'$\hat{\rm{MF}}^{\rm QE}$', cmap='YlGnBu', hold=True, min=-5e15, max=5e15)
        plt.axes(ax[3])
        hp.mollview(hp.alm2map(bp(anass[0][0].mf,1,20), nside=256), title='cscomp.{fg} - '.format(fg=ana.fg)+r'$\hat{\rm{MF}}^{\rm QE}$ - bandpassed', cmap='YlGnBu', hold=True, min=-0.01, max=0.01)
        plt.show()

In [ ]:
for anasi, anas in enumerate(anass):
    for anai, ana in enumerate(anas):
        rotation = np.array([40,-55])
        lonra = np.array([-45,-42.5])
        latra = np.array([52.5,55])
        fig, ax = plt.subplots(1, 2, figsize=(24,12))
        # plt.axes(ax[0])
        # hp.cartview(hp.alm2map(scale_and_bandpass(anass[0][0].plms_in[-1]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - input deflection'.format(fg=ana.fg), cmap="YlGnBu", hold=True)#, min=-0.002, max=0.002)
        plt.axes(ax[0])
        hp.cartview(hp.alm2map(scale_and_bandpass(ana.plms[-1,0]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - QE estimate'.format(fg=ana.fg), cmap="YlGnBu", hold=True, min=-0.0009, max=0.0009)
        plt.axes(ax[1])
        hp.cartview(hp.alm2map(scale_and_bandpass(ana.plms[-1,-1]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - MAP estimate'.format(fg=ana.fg), cmap="YlGnBu", hold=True, min=-0.0009, max=0.0009)
        fig.savefig('plots/deflection_225x225_cscomp.{fg}_QEMAP.png'.format(fg=fg), bbox_inches='tight')

In [ ]:
# Wide view
for anasi, anas in enumerate(anass):
    for anai, ana in enumerate(anas):
        # if anai == 0:
            rotation = np.array([40,-60])
            lonra = np.array([-35,35])
            latra = np.array([-15,27])
            # plt.axes(ax[0])
            
            fig, ax = plt.subplots(1, 1, figsize=(12,12))
            hp.cartview(hp.alm2map(scale_and_bandpass(anass[0][0].plms_in[-1]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - input'.format(fg=ana.fg), cmap="YlGnBu", hold=True)#, min=-0.002, max=0.002)
            fig.savefig('plots/deflection_fullpatch_cscomp{fg}_IN.png'.format(fg=ana.fg), bbox_inches='tight')
            
            # plt.axes(ax[0])
            # hp.cartview(hp.alm2map(scale_and_bandpass(ana.plms[-1,0]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp{fg} - QE estimate deflection'.format(fg=ana.fg), cmap="YlGnBu", hold=True)#, min=-0.002, max=0.002)
            fig, ax = plt.subplots(1, 1, figsize=(12,12))
            # plt.axes(ax[0])
            hp.cartview(hp.alm2map(scale_and_bandpass(ana.plms[-1,0]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - QE estimate'.format(fg=ana.fg), cmap="YlGnBu", hold=True)#, min=-0.002, max=0.002)
            fig.savefig('plots/deflection_fullpatch_cscomp{fg}_QE.png'.format(fg=ana.fg), bbox_inches='tight')
            
            fig, ax = plt.subplots(1, 1, figsize=(12,12))
            # plt.axes(ax[0])
            hp.cartview(hp.alm2map(scale_and_bandpass(ana.plms[-1,-1]), nside=2048), lonra=lonra, latra=latra, rot=rotation, title='cscomp.{fg} - MAP estimate'.format(fg=ana.fg), cmap="YlGnBu", hold=True)#, min=-0.002, max=0.002)
            fig.savefig('plots/deflection_fullpatch_cscomp{fg}_MAP.png'.format(fg=ana.fg), bbox_inches='tight')

In [ ]:
for anasi, anas in enumerate(anass):
    fig, ax = plt.subplots(1,3, figsize=(22,6), sharey=True)
    for anai, ana in enumerate(anas):
        ax[anai].plot(scale_factor_L*ana.cpp, label=r'$C_L^{\phi\phi,{\rm fid}}$', color='black', lw=3)
        for simidxi, simidx in enumerate(ana.simidxs):
            st = '' if simidx == 0 else '_'
            plms_QE_WF = np.copy(ana.plms[simidxi,0])
            anass[0][0].WF[0] = 1.
            almxfl(plms_QE_WF, np.nan_to_num(1/anass[0][0].WF), ana.lm_max_qlm[1], True) # Wiener-filter QE
            ax[anai].plot(scale_factor_L*hp.alm2cl(plms_QE_WF)/np.mean(anas[0].mask), label=st+r'$C_L^{\hat{\phi}^{\rm QE}\hat{\phi}^{\rm QE}}$', color='orange', alpha=0.5)
            for itmax in range(ana.itmax):
                st = '' if itmax == 0 else '_'
                ax[anai].plot(scale_factor_L*hp.alm2cl(ana.plms[simidxi,itmax])/np.mean(anas[0].mask), label=st+r'$C_L^{\hat{\phi}^{\rm MAP, WF}\hat{\phi}^{\rm MAP, WF}}$', color='blue', alpha=0.5)

            ax[anai].plot(scale_factor_L*hp.alm2cl(anas[0].mf), label=st+r'$C_L^{\hat{\phi}^{\rm MF}\hat{\phi}^{\rm MF}}$', color='cyan')
            ax[anai].plot(scale_factor_L*hp.alm2cl(ana.plms[simidxi,0]/np.sqrt(np.mean(anas[0].mask))-anas[0].mf), label=st+r'$C_L^{\hat{\phi}^{\rm QE+MF}\hat{\phi}^{\rm QE+MF}}$', color='green')

        ax[anai].legend(fontsize=18, loc='upper right', title='cscomp.{fg}'.format(fg=ana.fg))
        ax[anai].set_xlabel('Multipole, L', fontsize=14)
        ax[anai].set_ylabel(r'$\left[L(L+1)\right]^2/(2\pi)$', fontsize=14)
        ax[anai].set_xscale('log')
        ax[anai].set_xlim(1,3000)
        ax[anai].set_ylim(1e-9,1e-6)
        ax[anai].set_yscale('log')